In [1]:
!python prepare_all_tensors.py

CONFIGS --------
reading data from dir: D:\Projects\DeepLearningForPCAP\data\isot_app_and_botnet_dataset\all_filtered_dns\
using max packet count: 50
saving resulting tesnors to dir: D:\Projects\DeepLearningForPCAP\data\tensors\
WORKING ----
... extracting packet data ...
... padding packet tensors ...
... one hot encoding transport layer and traffic data ...
RESULTS----
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\pcap_tensor.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\traffic_data.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\traffic_one_hot.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\traffic_indices.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\transport_layer_list.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\transport_one_hot.npy
.... saved D:\Projects\DeepLearningForPCAP\data\tensors\transport_indices.npy


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd


import tensorflow as tf
import keras
from keras import layers


In [3]:
with open("D:\\Projects\DeepLearningForPCAP\\data\\tensors\\pcap_tensor.npy", 'rb') as f:
    pcap_tensor = np.load(f)

with open("D:\\Projects\DeepLearningForPCAP\\data\\tensors\\transport_one_hot.npy", 'rb') as f:
    transport_one_hot = np.load(f)

with open("D:\\Projects\DeepLearningForPCAP\\data\\tensors\\traffic_one_hot.npy", 'rb') as f:
    traffic_one_hot = np.load(f)


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(pcap_tensor, traffic_one_hot, test_size = 0.2, random_state=12)


## Define the CNN Model

In [11]:
kernel_size = 4
input_shape = pcap_tensor.shape[1]
output_shape = traffic_one_hot.shape[1]

n_training_epochs = 5
batch_size = 64

In [12]:
def base_classifier_model(input_shape, output_shape, conv_kernel_size, n_conv_layers):
    """ Defines the baseline classifier model with 3 Convulutional Layers aand 2 max pool layers"""
    model = keras.models.Sequential()

    for _ in range(n_conv_layers):
        model.add(layers.Conv1D(32, (conv_kernel_size), activation='relu', input_shape=(input_shape, 1)))
        model.add(layers.MaxPooling1D(2))



    model.add(layers.Conv1D(64, conv_kernel_size, activation='relu'))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(output_shape, activation='softmax'))

    return model


model = base_classifier_model(input_shape, output_shape, kernel_size, 3)



In [16]:
param_list = [
    {'kernel_size': 3, 'n_conv_layers': 2},
    {'kernel_size': 3, 'n_conv_layers': 3},
    {'kernel_size': 3, 'n_conv_layers': 4},
    # {'kernel_size': 3, 'n_conv_layers': 5},

    # {'kernel_size': 4, 'n_conv_layers': 2},
    # {'kernel_size': 4, 'n_conv_layers': 3},
    # {'kernel_size': 4, 'n_conv_layers': 4},
    # {'kernel_size': 4, 'n_conv_layers': 5},

    # {'kernel_size': 5, 'n_conv_layers': 2},
    # {'kernel_size': 5, 'n_conv_layers': 3},
    # {'kernel_size': 5, 'n_conv_layers': 4},
    # {'kernel_size': 5, 'n_conv_layers': 5}
]

results = []

for using_param in param_list:
    model = base_classifier_model(input_shape, output_shape, using_param['kernel_size'], using_param['n_conv_layers'])

    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


    model.fit(X_train, Y_train, epochs=n_training_epochs, batch_size=batch_size, verbose=0)
    test_loss, test_acc = model.evaluate(X_test, Y_test)

    results.append({'test_loss': test_loss, 'test_acc': test_acc})

3/3 [==============================] - 0s 6ms/step - loss: 1.6837 - accuracy: 0.6286


In [17]:
results

[{'test_loss': 1.3214871883392334, 'test_acc': 0.7428571581840515},
 {'test_loss': 1.3062856197357178, 'test_acc': 0.6285714507102966},
 {'test_loss': 1.6837176084518433, 'test_acc': 0.6285714507102966}]

In [40]:
outtable = pd.DataFrame()

for p, r in zip(param_list, results):
    p.update(r)
    outtable = pd.concat([outtable, pd.DataFrame(p, index=[0])])

outtable.sort_values(['test_acc'], ascending=False)

,kernel_size,n_conv_layers,test_loss,test_acc
0,3,2,1.321487,0.742857
0,3,3,1.306286,0.628571
0,3,4,1.683718,0.628571
